<a href="https://colab.research.google.com/github/MaheepChaudhary/Cat_and_Dog_Audio_Classifier/blob/master/Cat_and_Dog_Audio_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir .kaggle

import json
#In username enter your username obtained from kaggle and key obtained
token = {	"username":"amratye","key":"b7620e5da4833304f5e9ec08b2daba2e"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
    
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d mmoreaux/audio-cats-and-dogs


mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
 83% 81.0M/97.0M [00:03<00:01, 16.2MB/s]
100% 97.0M/97.0M [00:03<00:00, 26.6MB/s]


In [3]:
!unzip /content/{/content}/datasets/mmoreaux/audio-cats-and-dogs/audio-cats-and-dogs.zip

Archive:  /content/{/content}/datasets/mmoreaux/audio-cats-and-dogs/audio-cats-and-dogs.zip
  inflating: cats_dogs/cat_1.wav     
  inflating: cats_dogs/cat_10.wav    
  inflating: cats_dogs/cat_100.wav   
  inflating: cats_dogs/cat_101.wav   
  inflating: cats_dogs/cat_102.wav   
  inflating: cats_dogs/cat_103.wav   
  inflating: cats_dogs/cat_105.wav   
  inflating: cats_dogs/cat_106.wav   
  inflating: cats_dogs/cat_107.wav   
  inflating: cats_dogs/cat_108.wav   
  inflating: cats_dogs/cat_109.wav   
  inflating: cats_dogs/cat_11.wav    
  inflating: cats_dogs/cat_110.wav   
  inflating: cats_dogs/cat_112.wav   
  inflating: cats_dogs/cat_113.wav   
  inflating: cats_dogs/cat_114.wav   
  inflating: cats_dogs/cat_115.wav   
  inflating: cats_dogs/cat_116.wav   
  inflating: cats_dogs/cat_117.wav   
  inflating: cats_dogs/cat_118.wav   
  inflating: cats_dogs/cat_119.wav   
  inflating: cats_dogs/cat_12.wav    
  inflating: cats_dogs/cat_120.wav   
  inflating: cats_dogs/cat_121.wav

In [4]:
#importing all libraries
import os
import IPython.display as ipd
import librosa
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Lambda,Input
from tensorflow.keras.activations import softmax,relu
from tensorflow.keras import backend as K
import numpy as np

In [35]:
def read(path):
  data_dog = [librosa.load(os.path.join(os.path.join(path,'dog'),x))[0] for x in os.listdir(os.path.join(path,'dog'))]
  label_dog = np.zeros_like(data_dog)
  data_cat = [librosa.load(os.path.join(os.path.join(path,'cat'),x))[0] for x in os.listdir(os.path.join(path,'cat'))]
  label_cat = np.ones_like(data_cat)
  return data_dog, data_cat, label_dog, label_cat

In [37]:
train_path = '/content/cats_dogs/train'
data_dog, data_cat, label_dog, label_cat = read(train_path)

In [77]:
#preocessing the dataset
def preprocess(data_dog,data_cat,label_dog,label_cat):
  #it takes time so we defined it under a function after running it one time we can comment out the 11th line as the values of data_dog,data_cat,lable_dog,label_cat are founded
  print(type(data_dog))
  #0.16 is the maximum value so we are normalizing the dataset
  data = np.concatenate((data_dog,data_cat),axis = 0)/1.16
  label = np.concatenate((label_dog,label_cat),axis = 0)
  train = []
  for i,j in zip(data,label):
    train.append([i,j])
  '''
  max_arr,min_arr = [],[]
  for i in data:
    max_arr.append(np.max(i))
    min_arr.append(np.min(i))    
  max = np.max(max_arr)
  min = np.min(min_arr)
  print("The minimum and the maximum values are {} and {}".format(max,min))
  '''
  return train

In [80]:
train = np.array(preprocess(data_dog, data_cat, label_dog, label_cat))
np.random.shuffle(train)
print(train)

<class 'list'>
[[array([-7.3730973e-07, -1.7804021e-05, -2.7146969e-05, ...,
        5.0115041e-03,  5.5791857e-03,  0.0000000e+00], dtype=float32)
  0]
 [array([0.        , 0.        , 0.        , ..., 0.03049052, 0.04158887,
       0.0285936 ], dtype=float32)
  1]
 [array([0.        , 0.        , 0.        , ..., 0.00460004, 0.00471852,
       0.        ], dtype=float32)
  1]
 [array([ 0.04726376,  0.0740867 ,  0.08507161, ..., -0.12593308,
       -0.15221879, -0.1042077 ], dtype=float32)
  1]
 [array([-0.0022927 , -0.00133908, -0.00162678, ..., -0.00252891,
       -0.00488138,  0.        ], dtype=float32)
  1]
 [array([0.        , 0.        , 0.        , ..., 0.0013524 , 0.00195667,
       0.        ], dtype=float32)
  0]
 [array([ 0.00329029,  0.00351053,  0.00135101, ..., -0.01266612,
       -0.01534169, -0.01040488], dtype=float32)
  1]
 [array([0.00036935, 0.00152783, 0.00440257, ..., 0.00086495, 0.00121879,
       0.        ], dtype=float32)
  0]
 [array([-2.0125293e-07, -1.430

In [ ]:
train = preprocess(train_path)
train_x,train_y = train[:,0]

(210,)
(210, 1)


ValueError: ignored

In [ ]:
print(train[0])

In [ ]:
print(train_x)
print(train_x.shape)

In [ ]:
print(train_y)
print(train_y.shape)

[1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 0
 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 1 1 0 1 0 0 1
 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0
 0 1 1 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 1 1 0 0 1 1 0 0 1 1 1
 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0
 1 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0]
(210,)


In [ ]:
|train_path = '/content/cats_dogs/train'
test_path = '/content/cats_dogs/test'
train = preprocess(train_path)
test = preprocess(test_path)  
train_x,train_y = train[0],train[1]
test_x,test_y = test[0],test[1]

NameError: ignored

In [ ]:
#preparing the base model
def base_model():
  model = Sequential()
  model.add(Dense(input_shape,activation = 'relu'))
  model.add(Dense(input_shape,activation = 'relu'))
  model.add(Dropout(0.1))
  model.add(Dense(input_shape,activation = 'relu'))
  model.add(Dropout(0.3))
  model.add(Dense(input_shape,activation = 'relu'))
  #model.add(Dropout(0.5))
  #model.add(Dense(input_shape,activation = 'relu'))
  return model
  

In [ ]:
input1 = 